In [ ]:
import numpy as np

from copy import deepcopy
from label_manipulation import median_filter_labels,fill_closest
from label_manipulation import fill_closest
from SignalRecognition.cwt_learner.wavelet_feature_engineering import CWT_learner
from signal_database.labelSelectedData import SignalDB,SignalBundle
import matplotlib.pyplot as plt
from plot_generator import plotResult_colorbars
import tensorflow as tf
from keras.layers import LSTM
import pandas as pd
from scipy.interpolate import interp1d
from keras.models import Sequential
from keras.layers import Dense
import pandas as pd
from keras.wrappers.scikit_learn import KerasClassifier

from keras.utils import Sequence
import numpy as np

sdb = SignalDB('grab_objects', path='./')
training_data_ = sdb.get_labeleddata()

%matplotlib

plt.figure()
for ii,example in enumerate(training_data_):
    plt.subplot(len(training_data_)*2,1,ii*2 + 1)
    for channel in example.signal_bundle.signals:
        plt.plot(channel)
    plt.subplot(len(training_data_)*2,1,ii*2 + 2)
    labels = example.labels
    plotResult_colorbars(labels, range(len(labels)))
plt.show()

all_possible_labels = []
for example in training_data_:
        all_possible_labels = all_possible_labels + example.labels
print set(all_possible_labels)


training_data_relabeled = deepcopy(training_data_)

# label map 
label_map = {
    "freespace" : "freespace",
    "grab_hex" : "grab",
    "grab_cup" : "grab",
    "grab_foamblock" : "grab",
    "release_foamblock" : "release",
    "release_cup" : "release",
    "release_hex" : "release",
    "hold_cup" : "hold",
    "hold_hex" : "hold",
    "hold_foamblock" : "hold",
    "hand_on" : "hand_on",
    "hand_off" : "hand_off",
    "still" : "still",
    "" : ""
}



for example in training_data_relabeled:
        example.labels = [label_map[label] for label in example.labels]

cwt_learn_training = CWT_learner(signal_indices = [0,1,2,3])
training_data = training_data_relabeled[0:10]
testing_data = training_data_relabeled[10:15]
for ld in training_data:
    labels = [label for label in ld.labels]
    signals = []
    signals.append(list(np.array(ld.signal_bundle.signals[0]) +  np.array(ld.signal_bundle.signals[1]))) 
    signals.append(ld.signal_bundle.signals[2])
    
    signals.append(list(np.array(ld.signal_bundle.signals[3]) +  np.array(ld.signal_bundle.signals[4]))) 
    signals.append(ld.signal_bundle.signals[5])    
    
    cwt_learn_training.add_training_data(signals,labels)

(trainXCWT,trainYCWT) = cwt_learn_training.get_examples_with_weights(cwt_learn_training.training_data_sets,with_window = "Off",
                                                         signal_indices = cwt_learn_training.signal_indices
                                                         ,wavelet = cwt_learn_training.wavelet)

all_labels = []
for example in training_data_relabeled:
        all_labels = example.labels + all_labels
label_types = list(set(all_labels))
print label_types

cwt_learn_test = CWT_learner(signal_indices = [0,1,2,3])
training_data = training_data_relabeled[0:10]
testing_data = training_data_relabeled[10:15]
for ld in testing_data:
    labels = [label for label in ld.labels]
    signals = []
    signals.append(list(np.array(ld.signal_bundle.signals[0]) +  np.array(ld.signal_bundle.signals[1]))) 
    signals.append(ld.signal_bundle.signals[2])
    
    signals.append(list(np.array(ld.signal_bundle.signals[3]) +  np.array(ld.signal_bundle.signals[4]))) 
    signals.append(ld.signal_bundle.signals[5])    
    
    cwt_learn_test.add_training_data(signals,labels)
    
(testXCWT,testYCWT) = cwt_learn_test.get_examples_with_weights(cwt_learn_test.training_data_sets,with_window = "Off",
                                                         signal_indices = cwt_learn_test.signal_indices
                                                         ,wavelet = cwt_learn_test.wavelet)


In [ ]:

trainX = np.array(trainXCWT)
trainY = np.array(trainYCWT)
testX = np.array(testXCWT)
testY = np.array(testYCWT)


trainY = fill_closest(trainY)
testY = fill_closest(testY)


Y_train_one_hot = pd.get_dummies(trainY)
Y_test_one_hot = pd.get_dummies(testY)

label_names_df = list(Y_train_one_hot.columns.values)

# wavelet + signal augmentation
signal_train_X = np.array([[]]).reshape(0,len(cwt_learn_training.signal_indices))
signal_train_Y = np.array([])
for ii in range(len(cwt_learn_training.training_data_sets)):
    signals = np.transpose(cwt_learn_training.training_data_sets[ii].signal_bundle.signals)
    labels = cwt_learn_training.training_data_sets[ii].labels
    labels = fill_closest(labels)
    signal_train_X = np.append(signal_train_X,signals,axis = 0)
    signal_train_Y = np.append(signal_train_Y,labels)
    
    
signal_test_X = np.array([[]]).reshape(0,len(cwt_learn_test.signal_indices))
signal_test_Y = np.array([])
for ii in range(len(cwt_learn_test.training_data_sets)):
    signals = np.transpose(cwt_learn_test.training_data_sets[ii].signal_bundle.signals)
    labels = cwt_learn_test.training_data_sets[ii].labels
    labels = fill_closest(labels)
    signal_test_X = np.append(signal_test_X,signals,axis = 0)
    signal_test_Y = np.append(signal_test_Y,labels)


trainX = np.append(trainX,signal_train_X,axis = 1)
testX = np.append(testX,signal_test_X,axis = 1)


In [51]:
# computing sample weights
from sklearn.utils.class_weight import compute_sample_weight
sample_weights = compute_sample_weight('balanced',trainY)

In [71]:
## trying LSTM
# training set: full_train_X, full_Y_one_hot -> the order is correct

def LSTM_model(data_dim,timesteps,batch_size,num_classes,stateful = True,
               sample_weight_mode = None):
    # expected input data shape: (batch_size, timesteps, data_dim)
    model_LSTM = Sequential()
    model_LSTM.add(LSTM(50, return_sequences=True,
                   input_shape=(timesteps, data_dim),batch_size = batch_size
                        ,stateful=stateful))  # returns a sequence of vectors of dimension 32
    model_LSTM.add(LSTM(50,stateful=stateful))  # return a single vector of dimension 32
    model_LSTM.add(Dense(50, activation='relu'))
    # model_LSTM.add(Dense(10, activation='relu'))
    model_LSTM.add(Dense(num_classes, activation='softmax'))
    model_LSTM.compile( loss='categorical_crossentropy',
                        optimizer='rmsprop',
                        metrics=['accuracy'],
                        sample_weight_mode=sample_weight_mode)
    return model_LSTM
    
    
data_dim = np.shape(trainX)[1]
timesteps = 100
batch_size = 100
num_classes = len(label_names_df)

model_LSTM = LSTM_model(data_dim,timesteps,batch_size,num_classes)
model_LSTM_nostateful = LSTM_model(data_dim,timesteps,batch_size,num_classes,stateful=False)
    

In [54]:



class SeqGen(Sequence):
    def __init__(self, x_set, y_set, batch_size, window_size,batch_stride = 1,
                 sample_weights = None):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size
        self.window_size = window_size
        self.batch_stride = batch_stride
        self.sw = sample_weights
            

    def __len__(self):
        return len(self.x)//self.batch_stride

    def __getitem__(self, idx):
        batch_x = []
        batch_y = []

        current_x = np.roll(self.x, idx*self.batch_stride, axis=0)
        current_y = np.roll(self.y, idx*self.batch_stride, axis=0)
        
        for ii in range(self.batch_size):
            batch_x.append(np.roll(current_x, ii,axis = 0)[:self.window_size])
            batch_y.append(np.roll(current_y, ii,axis = 0)[:self.window_size][-1])
        
        if self.sw is None:
            return np.array(batch_x), np.array(batch_y)
        
        else:
            batch_sw = []
            current_sw = np.roll(self.sw, idx*self.batch_stride, axis=0)
            for ii in range(self.batch_size):    
                batch_sw.append(np.roll(current_sw, ii,axis = 0)[:self.window_size][-1])

            return np.array(batch_x), np.array(batch_y), np.array(batch_sw)
    


In [56]:
skip = 100
trainX_sub_sampled = trainX[0:-1:skip]
trainY_one_hot_sampled = Y_train_one_hot[0:-1:skip]
sw_train_sub_sampled = sample_weights[0:-1:skip]

testX_sub_sampled = full_test_X[0:-1:skip]
testY_one_hot_sampled = Y_test_one_hot[0:-1:skip]
sw_test_sub_sampled = sample_weights[0:-1:skip]

validationX_sub_sampled = testX[50:-1:skip]
validationY_one_hot_sampled = Y_test_one_hot[50:-1:skip]
sw_validation_sub_sampled = sample_weights[50:-1:skip]


data_dim = np.shape(full_train_X)[1]
timesteps = 100
window = 100
batch_stride = 100
batch_size = 100

sg_train = SeqGen(trainX_sub_sampled,trainY_one_hot_sampled,batch_size,window,
                  batch_stride=batch_stride,sample_weights=sw_train_sub_sampled)

sg_test = SeqGen(testX_sub_sampled,testY_one_hot_sampled,batch_size,window,
                  batch_stride=batch_stride, sample_weights=sw_test_sub_sampled)

sg_validation = SeqGen(validationX_sub_sampled,validationY_one_hot_sampled,batch_size,window,
                  batch_stride=batch_stride, sample_weights=sw_validation_sub_sampled)

In [59]:
# np.shape(sg_train[0][0])

(100, 100, 84)

In [75]:
model_LSTM.fit_generator(sg_train,validation_data=sg_validation,
                    epochs=10)


Epoch 1/5
 1/36 [..............................] - ETA: 6s - loss: 0.3461 - acc: 0.9000

 2/36 [>.............................] - ETA: 5s - loss: 0.4356 - acc: 0.8950

 3/36 [=>............................] - ETA: 5s - loss: 0.6547 - acc: 0.8600

 4/36 [==>...........................] - ETA: 5s - loss: 0.5009 - acc: 0.8950

 5/36 [===>..........................] - ETA: 5s - loss: 0.6173 - acc: 0.8400

...

32/36 [=========================>....] - ETA: 0s - loss: 0.3198 - acc: 0.8853

33/36 [==========================>...] - ETA: 0s - loss: 0.3122 - acc: 0.8870

34/36 [===========================>..] - ETA: 0s - loss: 0.3085 - acc: 0.8888

35/36 [============================>.] - ETA: 0s - loss: 0.3893 - acc: 0.8803

36/36 [==============================] - 4s 103ms/step - loss: 0.4556 - acc: 0.8683 - val_loss: 0.8446 - val_acc: 0.7342


In [182]:
model_LSTM_nostateful.fit_generator(sg_train,validation_data=sg_validation,
                    epochs=1)
    

Epoch 1/1


 1/36 [..............................] - ETA: 8s - loss: 0.0087 - acc: 0.9900

 2/36 [>.............................] - ETA: 6s - loss: 0.0404 - acc: 0.9800

 3/36 [=>............................] - ETA: 5s - loss: 0.0359 - acc: 0.9833

 4/36 [==>...........................] - ETA: 5s - loss: 0.0270 - acc: 0.9875

...

32/36 [=========================>....] - ETA: 0s - loss: 0.0226 - acc: 0.9900

33/36 [==========================>...] - ETA: 0s - loss: 0.0225 - acc: 0.9900

34/36 [===========================>..] - ETA: 0s - loss: 0.0242 - acc: 0.9894

35/36 [============================>.] - ETA: 0s - loss: 0.0236 - acc: 0.9897

36/36 [==============================] - 5s 127ms/step - loss: 0.0237 - acc: 0.9897 - val_loss: 0.3156 - val_acc: 0.9226


In [124]:
# model_LSTM_nostateful.summary()
# model_LSTM.summary()

In [241]:
test_one_hot_predict_series = []
test_one_hot_truth_series = []

for ii in range(len(sg_test)):
    test_one_hot_predict = model_LSTM_nostateful.predict(sg_test[ii][0],batch_size=batch_size)
    test_one_hot_predict_df = pd.DataFrame(test_one_hot_predict,columns=label_names_df)
    test_one_hot_predict_series_batch = list(test_one_hot_predict_df.idxmax(axis = 1))
    test_one_hot_predict_series = test_one_hot_predict_series + test_one_hot_predict_series_batch 
    
    test_one_hot_truth = (sg_test[ii][1] > 0.9).astype(int)
    test_one_hot_truth_df = pd.DataFrame(test_one_hot_truth,columns=label_names_df)
    test_one_hot_truth_series_batch = list(test_one_hot_truth_df.idxmax(axis = 1))
    test_one_hot_truth_series = test_one_hot_truth_series + test_one_hot_truth_series_batch

plt.figure(1)
ax = plt.subplot(2,1,1)

plotResult_colorbars(test_one_hot_predict_series, range(len(test_one_hot_predict_series)),
                     labelNames=label_types,ax=ax,medfiltwidth=1)
ax.set_xlim([0,len(test_one_hot_predict_series)])
ax = plt.subplot(2,1,2)

plotResult_colorbars(test_one_hot_truth_series, range(len(test_one_hot_truth_series)),
                     labelNames=label_types,medfiltwidth=1)
ax.set_xlim([0,len(test_one_hot_truth_series)])


print set(test_one_hot_truth_series)
print set(test_one_hot_predict_series)


set(['freespace', 'still', 'hand_off', 'hand_on', 'release', 'grab', 'hold'])
set(['freespace', 'still', 'hand_off', 'hand_on', 'release', 'grab', 'hold'])


AttributeError: 'function' object has no attribute 'summary'

In [222]:
def LSTM_model_no_batch(data_dim,timesteps,num_classes,stateful = True,
               sample_weight_mode = None):
    # expected input data shape: (batch_size, timesteps, data_dim)
    model_LSTM = Sequential()
    model_LSTM.add(LSTM(50, return_sequences=True,
                   input_shape=(timesteps, data_dim)
                        ,stateful=stateful))  # returns a sequence of vectors of dimension 32
    model_LSTM.add(LSTM(50,stateful=stateful))  # return a single vector of dimension 32
    model_LSTM.add(Dense(50, activation='relu'))
    # model_LSTM.add(Dense(10, activation='relu'))
    model_LSTM.add(Dense(num_classes, activation='softmax'))
    model_LSTM.compile( loss='categorical_crossentropy',
                        optimizer='rmsprop',
                        metrics=['accuracy'],
                        sample_weight_mode=sample_weight_mode)
    return model_LSTM
batch_size = 100

data_dim = 4
model_LSTM_no_wavelet = LSTM_model_no_batch(data_dim,timesteps,num_classes,stateful=False)
sg_train_no_wavelet = SeqGen(trainX_sub_sampled[:,80:],trainY_one_hot_sampled,batch_size,window,
                  batch_stride=batch_stride,sample_weights=sw_train_sub_sampled)

sg_test_no_wavelet = SeqGen(testX_sub_sampled[:,80:],testY_one_hot_sampled,batch_size,window,
                  batch_stride=batch_stride, sample_weights=sw_test_sub_sampled)

sg_validation_no_wavelet = SeqGen(validationX_sub_sampled[:,80:],validationY_one_hot_sampled,batch_size,window,
                  batch_stride=batch_stride, sample_weights=sw_validation_sub_sampled)

In [237]:

model_LSTM_no_wavelet.fit_generator(sg_train_no_wavelet,validation_data=sg_validation_no_wavelet,
                    epochs=50)


In [238]:
sg_test_no_wavelet_single = SeqGen(testX_sub_sampled[:,80:],testY_one_hot_sampled,100,window,
                  batch_stride=batch_stride, sample_weights=sw_test_sub_sampled)


In [239]:
np.shape(sg_test_no_wavelet_single[0][0])

(100, 100, 4)

In [240]:
test_one_hot_predict_series = []
test_one_hot_truth_series = []

for ii in range(len(sg_test_no_wavelet_single)):
    test_one_hot_predict = model_LSTM_no_wavelet.predict(sg_test_no_wavelet_single[ii][0],batch_size = 100)
    test_one_hot_predict_df = pd.DataFrame(test_one_hot_predict,columns=label_names_df)
    test_one_hot_predict_series_batch = list(test_one_hot_predict_df.idxmax(axis = 1))
    test_one_hot_predict_series = test_one_hot_predict_series + test_one_hot_predict_series_batch 
    
    test_one_hot_truth = (sg_test_no_wavelet_single[ii][1] > 0.9).astype(int)
    test_one_hot_truth_df = pd.DataFrame(test_one_hot_truth,columns=label_names_df)
    test_one_hot_truth_series_batch = list(test_one_hot_truth_df.idxmax(axis = 1))
    test_one_hot_truth_series = test_one_hot_truth_series + test_one_hot_truth_series_batch

plt.figure(2)
ax = plt.subplot(2,1,1)

plotResult_colorbars(test_one_hot_predict_series, range(len(test_one_hot_predict_series)),
                     labelNames=label_types,ax=ax,medfiltwidth=1)
ax.set_xlim([0,len(test_one_hot_predict_series)])
ax = plt.subplot(2,1,2)

plotResult_colorbars(test_one_hot_truth_series, range(len(test_one_hot_truth_series)),
                     labelNames=label_types,medfiltwidth=1)
ax.set_xlim([0,len(test_one_hot_truth_series)])

(0, 1900)

In [ ]:
# model_LSTM_no_wavelet.summary()
from keras.utils import plot_model
plot_model(model_LSTM_no_wavelet)

In [160]:
np.sum([len(testing_data[ii].signal_bundle.timestamps[0:-1:skip]) for ii in range(len(testing_data))])

1991

In [165]:
resampled_labels_truth = np.repeat(test_one_hot_predict_series,100)
plotResult_colorbars(resampled_labels_truth, range(len(resampled_labels_truth)),
                     labelNames=label_types,medfiltwidth=1)